In [2]:
import numpy as np
import pandas as pd 
from scipy.optimize import least_squares
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set(rc={"figure.figsize":(25, 10)})

In [4]:
dQI = pd.read_csv("DataForImpact/BIGdQI.csv")

In [5]:
dQI = dQI.loc[dQI["Loses"] > 0.]

In [6]:
dQI['Time'] = dQI['Time'].astype(float)
MI = np.array(dQI['Loses'])
dQ= np.array(dQI['volume'])
T = np.array(dQI['Time'], dtype=float)

In [7]:
T

array([4.710000e+02, 5.740000e+02, 6.030000e+02, ..., 1.499050e+06,
       1.499824e+06, 1.499825e+06])

### Usual OWM: $I_{t+1} = \rho I_t + \lambda Q_{t+1}$

In [15]:
cvlen = 35000
start_point = 1000

def fun1(x, mi: np.array, mi_prev: np.array, dq: np.array):
            return x[0] * mi_prev + x[1] * dq - mi

errorsSOWM = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(MI[1:i], MI[:i - 1], dQ[1:i]))
    errorsSOWM[i - start_point] = fun1(res_lsq01.x, MI[i], MI[i - 1], dQ[i])


print("MAE: ", np.mean(abs(errorsSOWM)))

MAE:  1.5680194241734309


In [16]:
cvlen = 35000
learning_window = 100
start_point = 1000

errorsSOWM_W = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq02 = least_squares(fun1, np.array([0.1, 0.1]), args=(MI[i - learning_window:i], MI[i - 1 - learning_window:i - 1], dQ[i - learning_window:i]))
    errorsSOWM_W[i - start_point] = fun1(res_lsq02.x, MI[i], MI[i - 1], dQ[i])

print("MAE: ", np.mean(abs(errorsSOWM_W)))

MAE:  1.3953206206925255


### SR model: $I_t = C \sqrt{Q_t}$

In [17]:
cvlen = 35000
start_point = 1000

def fun2(x, mi: np.array, dq: np.array):
            return x[0] * np.sqrt(dq) - mi

errorsSR = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq11 = least_squares(fun2, np.array([0.1]), args=(MI[1:i], dQ[1:i]))
    errorsSR[i - start_point] = fun2(res_lsq11.x, MI[i], dQ[i])


print("MAE: ", np.mean(abs(errorsSR)))

MAE:  2.1225440510941165


In [18]:
cvlen = 35000
learning_window = 100
start_point = 1000

errorsSR_W = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq12 = least_squares(fun2, np.array([0.1, 0.1]), args=(MI[i - learning_window:i], dQ[i - learning_window:i]))
    errorsSR_W[i - start_point] = fun2(res_lsq12.x, MI[i], dQ[i])

print("MAE: ", np.mean(abs(errorsSR_W)))

MAE:  1.5385424276542083


### Experimrntal Combo of SRM and OWM: $I_{t+1} = \rho I_t + \lambda \sqrt{Q_{t+1}}$

In [19]:
cvlen = 35000
start_point = 1000

def fun3(x, mi: np.array, mi_prev: np.array, dq: np.array):
            return x[0] * mi_prev + x[1] * np.sqrt(dq) - mi

errorsOWSR = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun3, np.array([0.1, 0.1]), args=(MI[1:i], MI[:i - 1], dQ[1:i]))
    errorsOWSR[i - start_point] = fun3(res_lsq01.x, MI[i], MI[i - 1], dQ[i])


print("MAE: ", np.mean(abs(errorsOWSR)))

MAE:  1.9932173237366233


In [20]:
cvlen = 35000
learning_window = 100
start_point = 1000

errorsOWSR_W = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq02 = least_squares(fun3, np.array([0.1, 0.1]), args=(MI[i - learning_window:i], MI[i - 1 - learning_window:i - 1], dQ[i - learning_window:i]))
    errorsOWSR_W[i - start_point] = fun3(res_lsq02.x, MI[i], MI[i - 1], dQ[i])

print("MAE: ", np.mean(abs(errorsOWSR_W)))

MAE:  1.5543257789006262


### Experimental model from OWM: $\frac{I_{i+1} - I_i}{\Delta t _{i+1}} = \rho I_i + \lambda \frac{Q_{i+1}}{\Delta t _{i+1}}$

In [9]:
dt = (T[1:] - T[:-1])
dIdt = (MI[1:] - MI[:-1]) / dt
Qdt = dQ[1:] / dt
Ii = MI[:-1]

In [22]:
cvlen = 35000
start_point = 1000

def fun3(x, dIdt: np.array, mi_prev: np.array, Qdt: np.array):
            return x[0] * mi_prev + x[1] * Qdt - dIdt

errorsOWC = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun3, np.array([0.1, 0.1]), args=(dIdt[:i], Ii[:i], Qdt[:i]))
    errorsOWC[i - start_point] = Ii[i] * (1 + dt[i] * res_lsq01.x[0]) + res_lsq01.x[1] * Qdt[i] - Ii[i + 1]


print("MAE: ", np.mean(abs(errorsOWC)))

MAE:  10.919683027663334


In [15]:
cvlen = 35000
learning_window = 1000
start_point = 1000
def fun3(x, dIdt: np.array, mi_prev: np.array, Qdt: np.array):
            return x[0] * mi_prev + x[1] * Qdt - dIdt

errorsOWC_W = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun3, np.array([0.1, 0.1]), args=(dIdt[i - learning_window:i], Ii[i - learning_window:i], Qdt[i - learning_window:i]))
    errorsOWC_W[i - start_point] = Ii[i] * (1 + dt[i] * res_lsq01.x[0]) + res_lsq01.x[1] * Qdt[i] - Ii[i + 1]


print("MAE: ", np.mean(abs(errorsOWC_W)))

MAE:  13.21009705233322


### Experimental model from our intuition: $\frac{y_{i + 1} - y_{i}}{\Delta t_{i+1}} = \rho y_i + \lambda$

In [24]:
y = MI / dQ

In [27]:
cvlen = 35000
start_point = 1000

def fun1(x, y: np.array, y_prev: np.array, dt: np.array):
            return y_prev * (1 + x[0] * dt) + x[1] * dt - y

errorsOWCS = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(y[1:i], y[:i - 1], dt[1:i]))
    errorsOWCS[i - start_point] = fun1(res_lsq01.x, y[i], y[i - 1], dt[i]) * dQ[i]


print("MAE: ", np.mean(abs(errorsOWCS)))

MAE:  4.623520484527892


In [30]:
cvlen = 35000
learning_window = 500
start_point = 2000

def fun1(x, y: np.array, y_prev: np.array, dt: np.array):
            return y_prev * (1 + x[0] * dt) + x[1] * dt - y

errorsOWCS_W = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(y[i - learning_window:i], y[i - 1 - learning_window:i - 1], dt[i - learning_window:i]))
    errorsOWCS_W[i - start_point] = fun1(res_lsq01.x, y[i], y[i - 1], dt[i]) * dQ[i]


print("MAE: ", np.mean(abs(errorsOWCS_W)))

MAE:  4.0392173162843426


### GOW model: $I_{t_{i+1}} = \rho ^{t_{i+1} - t_i} I_{t_i} + \lambda Q_{t_{i+1}}$

In [32]:
cvlen = 35000
start_point = 1000

def fun1(x, mi: np.array, mi_prev: np.array, dq: np.array, dt: np.array):
            return np.power(x[0], dt) * mi_prev + x[1] * dq - mi

errorsGOW = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(MI[1:i], MI[:i - 1], dQ[1:i], dt[1:i]))
    errorsGOW[i - start_point] = fun1(res_lsq01.x, MI[i], MI[i - 1], dQ[i], dt[i])


print("MAE: ", np.mean(abs(errorsGOW)))

MAE:  1.5712823852127695


In [10]:
cvlen = 35000
learning_window = 500
start_point = 1000

def fun1(x, mi: np.array, mi_prev: np.array, dq: np.array, dt: np.array):
            return np.power(x[0], dt) * mi_prev + x[1] * dq - mi

errorsGOW_W = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(MI[i - learning_window:i], MI[i - 1 - learning_window:i - 1], dQ[i - learning_window:i], dt[i - learning_window:i]))
    errorsGOW_W[i - start_point] = fun1(res_lsq01.x, MI[i], MI[i - 1], dQ[i], dt[i])


print("MAE: ", np.mean(abs(errorsGOW_W)))

MAE:  1.3665084586946301


### AR(1)

In [16]:
cvlen = 35000
start_point = 1000

def fun1(x, mi: np.array, mi_prev: np.array):
            return x[0] * mi_prev + x[1] - mi

errorsAR = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(MI[1:i], MI[:i - 1]))
    errorsAR[i - start_point] = fun1(res_lsq01.x, MI[i], MI[i - 1])


print("MAE: ", np.mean(abs(errorsAR)))

MAE:  2.08474538347663


In [17]:
cvlen = 35000
learning_window = 500
start_point = 1000

def fun1(x, mi: np.array, mi_prev: np.array):
            return x[0] * mi_prev + x[1] - mi

errorsAR_W = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(MI[i-learning_window:i], MI[i-1-learning_window:i - 1]))
    errorsAR_W[i - start_point] = fun1(res_lsq01.x, MI[i], MI[i - 1])


print("MAE: ", np.mean(abs(errorsAR_W)))

MAE:  1.6210042798402684
